In [1]:
import os
import sys
import csv
import wave
import copy
import math

import numpy as np
import pandas as pd

from sklearn.preprocessing import label_binarize
from sklearn.cross_validation import StratifiedKFold, KFold, train_test_split
from sklearn.svm import OneClassSVM, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop

sys.path.append("../")

from utilities.utils import *

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

from IPython.display import clear_output

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In [2]:
batch_size = 64
nb_feat = 34
nb_class = 4
nb_epoch = 80

optimizer = 'Adadelta'

In [3]:
params = Constants()
print(params)

----------------------------------------------------------------------------------------------------
available_emotions            ['ang' 'exc' 'neu' 'sad']                                             
conf_matrix_prefix            iemocap                                                               
framerate                     16000                                                                 
path_to_data                  /home/samarth/emotion_recognition-master/code/utilities/../../data/ses
path_to_features              /home/samarth/emotion_recognition-master/code/utilities/../../data/fea
sessions                      ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']          
types                         {1: <class 'numpy.int8'>, 2: <class 'numpy.int16'>, 4: <class 'numpy.i
----------------------------------------------------------------------------------------------------


In [4]:
import pickle
with open(params.path_to_data + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [5]:
data2[0]

{'a': 2.5,
 'd': 2.5,
 'emo_evo': [['neu'], ['neu'], ['neu'], ['neu']],
 'emotion': 'neu',
 'end': 8.2357,
 'id': 'Ses01F_impro01_F000',
 'mocap_hand': array([[        nan,         nan,         nan, ...,  -46.42036 ,
         -162.607345,  -80.458935],
        [        nan,         nan,         nan, ...,  -46.42206 ,
         -162.644865,  -80.503155],
        [        nan,         nan,         nan, ...,  -46.41771 ,
         -162.64734 ,  -80.53003 ],
        ..., 
        [        nan,         nan,         nan, ...,  -46.27356 ,
         -162.49055 ,  -79.90838 ],
        [        nan,         nan,         nan, ...,  -46.29963 ,
         -162.48961 ,  -79.8289  ],
        [        nan,         nan,         nan, ...,  -46.28651 ,
         -162.55255 ,  -79.82458 ]]),
 'mocap_head': array([[ -1.29297700e+01,  -1.35274400e+01,  -1.97916150e+01,
          -2.23723750e+01,  -1.35425840e+02,  -3.02012850e+01],
        [ -1.29120500e+01,  -1.35292750e+01,  -1.97787800e+01,
          -2.2348

In [ ]:
import pickle
with open(params.path_to_data + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [6]:
x_train2 = []
from sklearn.preprocessing import normalize
counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calc_feat.calculate_features(x_head, params.framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=78)
    x_train2.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train2 = np.array(x_train2)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [15]:
#x_train2 = x_train2.reshape(-1,78,34)
x_train2.shape

(4936, 78, 34)

In [47]:
def build_simple_lstm(nb_feat, nb_class, optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(78, nb_feat)))
    #model.add(Activation('tanh'))
    model.add(LSTM(256, return_sequences=False))
    #model.add(Activation('tanh'))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(nb_class))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [48]:
model = build_simple_lstm(nb_feat, nb_class)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 78, 512)           1120256   
_________________________________________________________________
activation_13 (Activation)   (None, 78, 512)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 256)               787456    
_________________________________________________________________
activation_14 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_15 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 2052      
__________

In [49]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = to_categorical(Y)

Y.shape

(4936, 4)

In [50]:
hist = model.fit(x_train2, Y, 
                 batch_size=batch_size, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 18s 5ms/step - loss: 1.3680 - acc: 0.3490 - val_loss: 1.3603 - val_acc: 0.3887
Epoch 2/80
3948/3948 [==============================] - 18s 4ms/step - loss: 1.3523 - acc: 0.3642 - val_loss: 1.3160 - val_acc: 0.4130
Epoch 3/80
3948/3948 [==============================] - 18s 4ms/step - loss: 1.3483 - acc: 0.3627 - val_loss: 1.3495 - val_acc: 0.3887
Epoch 4/80
3948/3948 [==============================] - 18s 4ms/step - loss: 1.3200 - acc: 0.3815 - val_loss: 1.2090 - val_acc: 0.4585
Epoch 5/80
3948/3948 [==============================] - 18s 4ms/step - loss: 1.2535 - acc: 0.4060 - val_loss: 1.2801 - val_acc: 0.4130
Epoch 6/80
3948/3948 [==============================] - 18s 4ms/step - loss: 1.2205 - acc: 0.4258 - val_loss: 1.2153 - val_acc: 0.4211
Epoch 7/80
3948/3948 [==============================] - 18s 4ms/step - loss: 1.1975 - acc: 0.4276 - val_loss: 1.2665 - val_acc: 0.4190
Epoch 8/

Epoch 61/80
3948/3948 [==============================] - 18s 4ms/step - loss: 0.6951 - acc: 0.7042 - val_loss: 1.4149 - val_acc: 0.4889
Epoch 62/80
3948/3948 [==============================] - 18s 4ms/step - loss: 0.6676 - acc: 0.7092 - val_loss: 1.4754 - val_acc: 0.4899
Epoch 63/80
3948/3948 [==============================] - 18s 4ms/step - loss: 0.6406 - acc: 0.7307 - val_loss: 1.4893 - val_acc: 0.5091
Epoch 64/80
3948/3948 [==============================] - 18s 4ms/step - loss: 0.6287 - acc: 0.7325 - val_loss: 1.5491 - val_acc: 0.5121
Epoch 65/80
3948/3948 [==============================] - 18s 4ms/step - loss: 0.6143 - acc: 0.7414 - val_loss: 1.7757 - val_acc: 0.4949
Epoch 66/80
3948/3948 [==============================] - 18s 4ms/step - loss: 0.6086 - acc: 0.7444 - val_loss: 1.6182 - val_acc: 0.4787
Epoch 67/80
3948/3948 [==============================] - 18s 4ms/step - loss: 0.5877 - acc: 0.7556 - val_loss: 1.5716 - val_acc: 0.5010
Epoch 68/80
3948/3948 [=========================

In [63]:
from features import stFeatureExtraction

    
def calculate_features2(frames, freq, options):
    window_sec = 0.02
    window_n = int(freq * window_sec)
    use_derivatives = False

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)
    #print (st_f.shape)
    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        if use_derivatives:
            deriv_st_f = np.zeros((st_f.shape[0]*3, i1 - i0), dtype=float)
        else:
            deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
            if use_derivatives:
                if st_f.shape[1] >= 2:
                    deriv_st_f[st_f.shape[0]:st_f.shape[0]*2, i - i0] = (st_f[:, i_right] - st_f[:, i_left]) / 2.
                    deriv_st_f[st_f.shape[0]*2:st_f.shape[0]*3, i - i0] = \
                        st_f[:, i] - 0.5*(st_f[:, i_left] + st_f[:, i_right])
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        if use_derivatives:
            deriv_st_f[st_f.shape[0]:st_f.shape[0]*2, 0] = st_f[:, 1] - st_f[:, 0]
            deriv_st_f[st_f.shape[0]*2:st_f.shape[0]*3, 0] = np.zeros(st_f.shape[0])
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        if use_derivatives:
            deriv_st_f[st_f.shape[0]:st_f.shape[0]*2, 0] = np.zeros(st_f.shape[0])
            deriv_st_f[st_f.shape[0]*2:st_f.shape[0]*3, 0] = np.zeros(st_f.shape[0])
        return deriv_st_f

In [64]:
x_train_sig2 = []
from sklearn.preprocessing import normalize
counter = 0
for ses_mod in data2:
    x_sig = ses_mod['signal']
    sig_avg = np.array_split(np.array(x_sig), 10)
    sig_feat = []
    for spl in sig_avg:
        #print(spl.shape)
        #print (x_sig.shape)
        st_features = calculate_features2(spl, params.framerate, None)
        st_features, _ = pad_sequence_into_array(st_features, maxlen=50)
        sig_feat.append( st_features ) 
    #st_features = calc_feat.calculate_features(x_sig, params.framerate, None)
    #st_features, _ = pad_sequence_into_array(st_features, maxlen=78)
    #x_train2.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    x_train_sig2.append(np.array(sig_feat))
    #break
    
x_train_sig2 = np.array(x_train_sig2)
x_train_sig2.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 10, 34, 50)

In [123]:
x_train3 = x_train2.reshape(-1,50,340)
x_train3.shape

(4936, 50, 340)

In [128]:
def build_simple_lstm2(nb_feat, nb_class, optimizer='SGD'):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(50,340)))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(nb_class))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [129]:
model = build_simple_lstm2(nb_feat, nb_class)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 50, 512)           1746944   
_________________________________________________________________
lstm_15 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_13 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_21 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 2052      
_________________________________________________________________
activation_22 (Activation)   (None, 4)                 0         
Total params: 2,668,036
Trainable params: 2,668,036
Non-trainable params: 0
_________________________________________________________________


In [130]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = to_categorical(Y)

Y.shape

(4936, 4)

In [131]:
hist = model.fit(x_train3, Y, 
                 batch_size=batch_size, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 14s 3ms/step - loss: 1.3609 - acc: 0.3341 - val_loss: 1.3501 - val_acc: 0.3684
Epoch 2/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3502 - acc: 0.3485 - val_loss: 1.3456 - val_acc: 0.3745
Epoch 3/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3461 - acc: 0.3503 - val_loss: 1.3467 - val_acc: 0.3887
Epoch 4/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3426 - acc: 0.3536 - val_loss: 1.3462 - val_acc: 0.3704
Epoch 5/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3407 - acc: 0.3620 - val_loss: 1.3484 - val_acc: 0.3856
Epoch 6/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3368 - acc: 0.3655 - val_loss: 1.3463 - val_acc: 0.3745
Epoch 7/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3341 - acc: 0.3658 - val_loss: 1.3400 - val_acc: 0.3897
Epoch 8/

KeyboardInterrupt: 

In [132]:
from keras.layers import Embedding, Conv2D, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution2D, GRU, MaxPooling1D


def build_simple_lstm3(nb_feat, nb_class, optimizer='Adam'):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(50,340)))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(512)))
    model.add(Activation('relu'))
    model.add(Dense(nb_class))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [133]:
model = build_simple_lstm2(nb_feat, nb_class)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 50, 512)           1746944   
_________________________________________________________________
lstm_17 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_15 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_23 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 2052      
_________________________________________________________________
activation_24 (Activation)   (None, 4)                 0         
Total params: 2,668,036
Trainable params: 2,668,036
Non-trainable params: 0
_________________________________________________________________


In [134]:
hist = model.fit(x_train3, Y, 
                 batch_size=batch_size, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 14s 4ms/step - loss: 1.3621 - acc: 0.3338 - val_loss: 1.3588 - val_acc: 0.3796
Epoch 2/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3503 - acc: 0.3523 - val_loss: 1.3544 - val_acc: 0.3735
Epoch 3/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3457 - acc: 0.3539 - val_loss: 1.3456 - val_acc: 0.3725
Epoch 4/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3412 - acc: 0.3627 - val_loss: 1.3421 - val_acc: 0.3937
Epoch 5/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3372 - acc: 0.3683 - val_loss: 1.3368 - val_acc: 0.3785
Epoch 6/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3333 - acc: 0.3685 - val_loss: 1.3458 - val_acc: 0.3968
Epoch 7/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.3279 - acc: 0.3678 - val_loss: 1.3373 - val_acc: 0.3745
Epoch 8/

Epoch 61/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.1247 - acc: 0.4899 - val_loss: 1.2518 - val_acc: 0.4211
Epoch 62/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.1393 - acc: 0.4868 - val_loss: 1.2819 - val_acc: 0.4332
Epoch 63/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.1279 - acc: 0.4785 - val_loss: 1.2472 - val_acc: 0.4352
Epoch 64/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.1347 - acc: 0.4739 - val_loss: 1.1984 - val_acc: 0.4595
Epoch 65/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.1237 - acc: 0.4833 - val_loss: 1.1776 - val_acc: 0.4666
Epoch 66/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.1244 - acc: 0.4830 - val_loss: 1.2235 - val_acc: 0.4332
Epoch 67/80
3948/3948 [==============================] - 12s 3ms/step - loss: 1.1173 - acc: 0.4894 - val_loss: 1.2429 - val_acc: 0.4150
Epoch 68/80
3948/3948 [=========================

In [138]:
x_train4 = []
from sklearn.preprocessing import normalize
counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calculate_features2(x_head, params.framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=400)
    x_train4.append( st_features )
    counter+=1
    if(counter%100==0):
        print(counter)
    #break
    
x_train4 = np.array(x_train4)
x_train4.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 400, 34)

In [152]:
x_train5 = x_train4.reshape(-1,34,400)

In [178]:
def build_simple_lstm3(nb_feat, nb_class, optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(34,400)))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(512)))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(nb_class))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [179]:
model = build_simple_lstm3(nb_feat, nb_class)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 34, 512)           1869824   
_________________________________________________________________
lstm_45 (LSTM)               (None, 34, 256)           787456    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 34, 512)           131584    
_________________________________________________________________
flatten_2 (Flatten)          (None, 17408)             0         
_________________________________________________________________
dense_41 (Dense)             (None, 256)               4456704   
_________________________________________________________________
activation_47 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 4)                 1028      
__________

In [180]:
hist = model.fit(x_train5, Y, 
                 batch_size=batch_size, nb_epoch=30, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 13s 3ms/step - loss: 1.3787 - acc: 0.3278 - val_loss: 1.3441 - val_acc: 0.3117
Epoch 2/30
3948/3948 [==============================] - 10s 2ms/step - loss: 1.3199 - acc: 0.3678 - val_loss: 1.2145 - val_acc: 0.4069
Epoch 3/30
3948/3948 [==============================] - 10s 2ms/step - loss: 1.2445 - acc: 0.4159 - val_loss: 1.1953 - val_acc: 0.4575
Epoch 4/30
3948/3948 [==============================] - 10s 2ms/step - loss: 1.1836 - acc: 0.4481 - val_loss: 1.2331 - val_acc: 0.3725
Epoch 5/30
3948/3948 [==============================] - 10s 2ms/step - loss: 1.1585 - acc: 0.4688 - val_loss: 1.2312 - val_acc: 0.4423
Epoch 6/30
3948/3948 [==============================] - 10s 2ms/step - loss: 1.1202 - acc: 0.4699 - val_loss: 1.3524 - val_acc: 0.3755
Epoch 7/30
3948/3948 [==============================] - 10s 2ms/step - loss: 1.1161 - acc: 0.4818 - val_loss: 1.1818 - val_acc: 0.4443
Epoch 8/

KeyboardInterrupt: 

In [7]:
text = []


for ses_mod in data2:
    text.append(ses_mod['transcription'])


In [10]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

In [11]:
MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
x_train_text = []

x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
import codecs
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

file_loc = params.path_to_data + '../glove.42B.300d.txt'

print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 2736 unique tokens
/home/samarth/emotion_recognition-master/code/utilities/../../data/sessions/../glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 90


In [13]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = to_categorical(Y)

Y.shape

(4936, 4)

In [14]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model_text.add(Convolution1D(256, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(128, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(64, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(32, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Flatten())
model_text.add(Dropout(0.2))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(LSTM(512, return_sequences=True, input_shape=(78, nb_feat)))

model_speech.add(LSTM(256, return_sequences=False))

model_speech.add(Dense(512))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech], mode='concat'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adadelta' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_combined.summary()

print("Model1 Built")


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 768)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               196864    
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 1028      
_________________________________________________________________
activation_6 (Activation)    (None, 4)                 0         
Total params: 7,514,448
Trainable params: 7,514,448
Non-trainable params: 0
_________________________________________________________________
Model1 Built


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [17]:
hist = model_combined.fit([x_train_text,x_train2], Y, 
                 batch_size=batch_size, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 33s 8ms/step - loss: 1.3603 - acc: 0.3490 - val_loss: 1.3286 - val_acc: 0.3897
Epoch 2/30
3948/3948 [==============================] - 23s 6ms/step - loss: 1.3350 - acc: 0.3680 - val_loss: 1.3343 - val_acc: 0.3877
Epoch 3/30
3948/3948 [==============================] - 23s 6ms/step - loss: 1.2788 - acc: 0.3967 - val_loss: 1.1852 - val_acc: 0.4960
Epoch 4/30
3948/3948 [==============================] - 23s 6ms/step - loss: 1.1708 - acc: 0.4742 - val_loss: 1.0295 - val_acc: 0.5891
Epoch 5/30
3948/3948 [==============================] - 23s 6ms/step - loss: 1.0439 - acc: 0.5491 - val_loss: 0.9774 - val_acc: 0.6093
Epoch 6/30
3948/3948 [==============================] - 23s 6ms/step - loss: 0.8913 - acc: 0.6345 - val_loss: 1.0757 - val_acc: 0.5719
Epoch 7/30
3948/3948 [==============================] - 23s 6ms/step - loss: 0.7926 - acc: 0.6849 - val_loss: 0.9587 - val_acc: 0.6103
Epoch 8/

KeyboardInterrupt: 

In [26]:

model_text2 = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text2.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model_text2.add(LSTM(512, return_sequences=True))
model_text2.add(Dropout(0.2))
model_text2.add(LSTM(256, return_sequences=False))
model_text2.add(Dropout(0.2))
model_text2.add(Dense(512))


model_speech = Sequential()
model_speech.add(LSTM(512, return_sequences=True, input_shape=(78, nb_feat)))
model_speech.add(Dropout(0.2))
model_speech.add(LSTM(256, return_sequences=False))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(512))

model_combined = Sequential()
model_combined.add(Merge([model_text2, model_speech], mode='concat'))
model_combined.add(Dropout(0.2))
model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adadelta' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_combined.summary()

print("Model1 Built")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_2 (Merge)              (None, 1024)              0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               262400    
_________________________________________________________________
activation_16 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 1028      
_________________________________________________________________
activation_17 (Activation)   (None, 4)                 0         
Total params: 5,707,888
Trainable params: 5,707,888
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [28]:
hist = model_combined.fit([x_train_text,x_train2], Y, 
                 batch_size=batch_size, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 146s 37ms/step - loss: 1.3477 - acc: 0.3513 - val_loss: 1.3413 - val_acc: 0.3360
Epoch 2/30
3948/3948 [==============================] - 144s 37ms/step - loss: 1.2763 - acc: 0.4108 - val_loss: 1.1672 - val_acc: 0.5040
Epoch 3/30
3948/3948 [==============================] - 144s 37ms/step - loss: 1.1915 - acc: 0.4770 - val_loss: 1.2037 - val_acc: 0.4524
Epoch 4/30
3948/3948 [==============================] - 144s 37ms/step - loss: 1.1174 - acc: 0.5228 - val_loss: 1.0894 - val_acc: 0.5304
Epoch 5/30
3948/3948 [==============================] - 145s 37ms/step - loss: 1.0456 - acc: 0.5522 - val_loss: 0.9770 - val_acc: 0.5860
Epoch 6/30
1856/3948 [=============>................] - ETA: 1:10 - loss: 1.0037 - acc: 0.5738

KeyboardInterrupt: 

In [84]:
model_speech = Sequential()
model_speech.add(Flatten(input_shape=(78, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.5))
model_speech.add(Dense(512))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.5))
model_speech.add(Dense(256))
model_speech.add(Activation('relu'))
model_speech.add(Dense(4))
model_speech.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_speech.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])
model_speech.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_26 (Flatten)         (None, 2652)              0         
_________________________________________________________________
dense_95 (Dense)             (None, 1024)              2716672   
_________________________________________________________________
activation_120 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_94 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_96 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_121 (Activation)  (None, 512)               0         
_________________________________________________________________
dropout_95 (Dropout)         (None, 512)               0         
__________

In [46]:
hist = model_speech.fit(x_train2, Y, 
                 batch_size=batch_size, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 2s 548us/step - loss: 2.1281 - acc: 0.2910 - val_loss: 1.3566 - val_acc: 0.3715
Epoch 2/80
3948/3948 [==============================] - 1s 153us/step - loss: 1.4061 - acc: 0.3311 - val_loss: 1.2927 - val_acc: 0.3877
Epoch 3/80
3948/3948 [==============================] - 1s 148us/step - loss: 1.2936 - acc: 0.3827 - val_loss: 1.2349 - val_acc: 0.4646
Epoch 4/80
3948/3948 [==============================] - 1s 147us/step - loss: 1.2313 - acc: 0.4126 - val_loss: 1.2053 - val_acc: 0.4777
Epoch 5/80
3948/3948 [==============================] - 1s 147us/step - loss: 1.2042 - acc: 0.4230 - val_loss: 1.1938 - val_acc: 0.4575
Epoch 6/80
3948/3948 [==============================] - 1s 148us/step - loss: 1.1894 - acc: 0.4367 - val_loss: 1.2200 - val_acc: 0.4545
Epoch 7/80
3948/3948 [==============================] - 1s 147us/step - loss: 1.1742 - acc: 0.4387 - val_loss: 1.2022 - val_acc: 0.4676
E

In [51]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model_text.add(Convolution1D(256, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(128, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(64, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(32, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Flatten())
model_text.add(Dropout(0.2))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(78, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.5))
model_speech.add(Dense(512))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.5))
model_speech.add(Dense(256))


model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech], mode='concat'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_speech.summary()
model_text.summary()
model_combined.summary()

print("Model1 Built")


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:37: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 2652)              0         
_________________________________________________________________
dense_60 (Dense)             (None, 1024)              2716672   
_________________________________________________________________
activation_70 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_50 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_61 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_71 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 512)               0         
__________

In [52]:
hist = model_combined.fit([x_train_text,x_train2], Y, 
                 batch_size=batch_size, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 7s 2ms/step - loss: 1.9704 - acc: 0.3009 - val_loss: 1.3321 - val_acc: 0.3957
Epoch 2/30
3948/3948 [==============================] - 5s 1ms/step - loss: 1.3356 - acc: 0.3627 - val_loss: 1.2091 - val_acc: 0.4717
Epoch 3/30
3948/3948 [==============================] - 5s 1ms/step - loss: 1.1706 - acc: 0.4742 - val_loss: 1.1535 - val_acc: 0.4727
Epoch 4/30
3948/3948 [==============================] - 5s 1ms/step - loss: 1.0260 - acc: 0.5537 - val_loss: 1.0815 - val_acc: 0.5263
Epoch 5/30
3948/3948 [==============================] - 5s 1ms/step - loss: 0.8625 - acc: 0.6408 - val_loss: 0.9700 - val_acc: 0.6255
Epoch 6/30
3948/3948 [==============================] - 5s 1ms/step - loss: 0.7373 - acc: 0.6988 - val_loss: 0.8648 - val_acc: 0.6599
Epoch 7/30
3948/3948 [==============================] - 5s 1ms/step - loss: 0.6145 - acc: 0.7523 - val_loss: 0.9981 - val_acc: 0.6447
Epoch 8/30
3948

KeyboardInterrupt: 

In [60]:
x_train_mocap = []
from sklearn.preprocessing import normalize
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    #x_normed = (x - x.min(0)) / x.ptp(0)
    #x_normed = x_normed - 0.5
    #x_normed[np.isnan(x)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(4936, 200, 189, 1)

In [61]:
from keras.layers import Conv2D

model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model_text.add(Convolution1D(256, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(128, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(64, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(32, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Flatten())
model_text.add(Dropout(0.2))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(78, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.5))
model_speech.add(Dense(512))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.5))
model_speech.add(Dense(256))

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(256))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech, model_mocap], mode='concat'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_speech.summary()
model_text.summary()
model_mocap.summary()
model_combined.summary()

print("Model1 Built")


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, padding="same", strides=(2, 2), input_shape=(200, 189,...)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: UserWarnin

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_22 (Flatten)         (None, 2652)              0         
_________________________________________________________________
dense_81 (Dense)             (None, 1024)              2716672   
_________________________________________________________________
activation_103 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_83 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_82 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_104 (Activation)  (None, 512)               0         
_________________________________________________________________
dropout_84 (Dropout)         (None, 512)               0         
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:57: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [62]:
hist = model_combined.fit([x_train_text,x_train2,x_train_mocap], Y, 
                 batch_size=batch_size, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 16s 4ms/step - loss: 2.7012 - acc: 0.3040 - val_loss: 1.3486 - val_acc: 0.3856
Epoch 2/30
3948/3948 [==============================] - 13s 3ms/step - loss: 1.3046 - acc: 0.3777 - val_loss: 1.2629 - val_acc: 0.3745
Epoch 3/30
3948/3948 [==============================] - 13s 3ms/step - loss: 1.0833 - acc: 0.5086 - val_loss: 1.0089 - val_acc: 0.5881
Epoch 4/30
3948/3948 [==============================] - 13s 3ms/step - loss: 0.8703 - acc: 0.6380 - val_loss: 0.9212 - val_acc: 0.6447
Epoch 5/30
3948/3948 [==============================] - 13s 3ms/step - loss: 0.6944 - acc: 0.7214 - val_loss: 0.9552 - val_acc: 0.6528oss - ETA: 1s - loss: 0.6922 - ac
Epoch 6/30
3948/3948 [==============================] - 13s 3ms/step - loss: 0.5642 - acc: 0.7844 - val_loss: 1.0067 - val_acc: 0.6680
Epoch 7/30
3948/3948 [==============================] - 13s 3ms/step - loss: 0.4611 - acc: 0.8260 - val_loss: 

In [65]:
x_train_sig2.shape

(4936, 10, 34, 50)

In [70]:
model_speech = Sequential()
model_speech.add(Flatten(input_shape=(10, 34, 50)))
model_speech.add(Dense(4096))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.25))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.25))
model_speech.add(Dense(256))
model_speech.add(Activation('relu'))
model_speech.add(Dense(4))
model_speech.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_speech.compile(loss='categorical_crossentropy',optimizer='adadelta' ,metrics=['acc'])
model_speech.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_25 (Flatten)         (None, 17000)             0         
_________________________________________________________________
dense_91 (Dense)             (None, 4096)              69636096  
_________________________________________________________________
activation_116 (Activation)  (None, 4096)              0         
_________________________________________________________________
dropout_92 (Dropout)         (None, 4096)              0         
_________________________________________________________________
dense_92 (Dense)             (None, 1024)              4195328   
_________________________________________________________________
activation_117 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_93 (Dropout)         (None, 1024)              0         
__________

In [71]:
hist = model_speech.fit(x_train_sig2, Y, 
                 batch_size=batch_size, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 10s 3ms/step - loss: 9.6804 - acc: 0.2999 - val_loss: 3.5498 - val_acc: 0.2895
Epoch 2/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.6456 - acc: 0.3032 - val_loss: 1.3203 - val_acc: 0.3806
Epoch 3/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.3653 - acc: 0.3422 - val_loss: 1.2881 - val_acc: 0.4059
Epoch 4/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.3274 - acc: 0.3650 - val_loss: 1.3396 - val_acc: 0.3745
Epoch 5/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.2815 - acc: 0.3875 - val_loss: 1.3551 - val_acc: 0.3917
Epoch 6/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.2341 - acc: 0.4093 - val_loss: 1.3132 - val_acc: 0.4474
Epoch 7/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.2096 - acc: 0.4177 - val_loss: 1.2791 - val_acc: 0.4180
Epoch 8/80
394

KeyboardInterrupt: 

In [76]:
counter = 0
for ses_mod in data2:
    if (ses_mod['id'][:5]=="Ses05"):
        break
    counter+=1
counter

3838

In [82]:
xtrain_sp = x_train2[:3838]
xtest_sp = x_train2[3838:]
ytrain_sp = Y[:3838]
ytest_sp = Y[3838:]

In [88]:
hist = model_speech.fit(xtrain_sp, ytrain_sp, 
                 batch_size=batch_size, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_data=(xtest_sp, ytest_sp))

Train on 3838 samples, validate on 1098 samples
Epoch 1/80
1024/3838 [=======>......................] - ETA: 0s - loss: 0.7506 - acc: 0.6641

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


3838/3838 [==============================] - 1s 170us/step - loss: 0.7892 - acc: 0.6501 - val_loss: 1.3314 - val_acc: 0.4763
Epoch 2/80
3838/3838 [==============================] - 1s 160us/step - loss: 0.8223 - acc: 0.6357 - val_loss: 1.3766 - val_acc: 0.4617
Epoch 3/80
3838/3838 [==============================] - 1s 154us/step - loss: 0.8114 - acc: 0.6428 - val_loss: 1.3192 - val_acc: 0.4763
Epoch 4/80
3838/3838 [==============================] - 1s 152us/step - loss: 0.8119 - acc: 0.6428 - val_loss: 1.3778 - val_acc: 0.4727
Epoch 5/80
3838/3838 [==============================] - 1s 150us/step - loss: 0.8213 - acc: 0.6285 - val_loss: 1.3790 - val_acc: 0.4909
Epoch 6/80
3838/3838 [==============================] - 1s 153us/step - loss: 0.8291 - acc: 0.6334 - val_loss: 1.3917 - val_acc: 0.4800
Epoch 7/80
3838/3838 [==============================] - 1s 151us/step - loss: 0.7647 - acc: 0.6717 - val_loss: 1.4769 - val_acc: 0.4927
Epoch 8/80
3838/3838 [==============================] - 1s 

Epoch 61/80
3838/3838 [==============================] - 1s 151us/step - loss: 0.6777 - acc: 0.6899 - val_loss: 1.6439 - val_acc: 0.5009
Epoch 62/80
3838/3838 [==============================] - 1s 152us/step - loss: 0.6635 - acc: 0.6939 - val_loss: 1.7320 - val_acc: 0.4936
Epoch 63/80
3838/3838 [==============================] - 1s 153us/step - loss: 0.6863 - acc: 0.6899 - val_loss: 1.6259 - val_acc: 0.4973
Epoch 64/80
3838/3838 [==============================] - 1s 153us/step - loss: 0.6867 - acc: 0.6899 - val_loss: 1.7540 - val_acc: 0.4991
Epoch 65/80
3838/3838 [==============================] - 1s 153us/step - loss: 0.6707 - acc: 0.6892 - val_loss: 1.8306 - val_acc: 0.4818
Epoch 66/80
3838/3838 [==============================] - 1s 154us/step - loss: 0.6473 - acc: 0.7095 - val_loss: 1.7406 - val_acc: 0.5036
Epoch 67/80
3838/3838 [==============================] - 1s 154us/step - loss: 0.6863 - acc: 0.6985 - val_loss: 1.6942 - val_acc: 0.4973
Epoch 68/80
3838/3838 [==================

In [89]:
x_train_long = []
from sklearn.preprocessing import normalize
counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calc_feat.calculate_features(x_head, params.framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=125)
    x_train_long.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_long = np.array(x_train_long)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [90]:
x_train_long.shape

(4936, 125, 34)

In [91]:
model_speech = Sequential()
model_speech.add(Flatten(input_shape=(125, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.5))
model_speech.add(Dense(512))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.5))
model_speech.add(Dense(256))
model_speech.add(Activation('relu'))
model_speech.add(Dense(4))
model_speech.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_speech.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])
model_speech.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_27 (Flatten)         (None, 4250)              0         
_________________________________________________________________
dense_99 (Dense)             (None, 1024)              4353024   
_________________________________________________________________
activation_124 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_96 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_100 (Dense)            (None, 512)               524800    
_________________________________________________________________
activation_125 (Activation)  (None, 512)               0         
_________________________________________________________________
dropout_97 (Dropout)         (None, 512)               0         
__________

In [92]:
hist = model_speech.fit(x_train_long, Y, 
                 batch_size=batch_size, nb_epoch=125, verbose=1, shuffle = True, 
                 validation_split = 0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/125
3948/3948 [==============================] - 3s 652us/step - loss: 2.1803 - acc: 0.2893 - val_loss: 1.3382 - val_acc: 0.3968
Epoch 2/125
3948/3948 [==============================] - 1s 186us/step - loss: 1.4264 - acc: 0.3546 - val_loss: 1.2773 - val_acc: 0.4200
Epoch 3/125
3948/3948 [==============================] - 1s 191us/step - loss: 1.2848 - acc: 0.4002 - val_loss: 1.2208 - val_acc: 0.4545
Epoch 4/125
3948/3948 [==============================] - 1s 191us/step - loss: 1.2313 - acc: 0.4151 - val_loss: 1.1997 - val_acc: 0.4686
Epoch 5/125
3948/3948 [==============================] - 1s 192us/step - loss: 1.2048 - acc: 0.4377 - val_loss: 1.2119 - val_acc: 0.4170
Epoch 6/125
3948/3948 [==============================] - 1s 189us/step - loss: 1.1881 - acc: 0.4372 - val_loss: 1.2104 - val_acc: 0.4484
Epoch 7/125
3948/3948 [==============================] - 1s 189us/step - loss: 1.1754 - acc: 0.4354 - val_loss: 1.1893 - val_acc: 0

3948/3948 [==============================] - 1s 192us/step - loss: 0.8648 - acc: 0.6102 - val_loss: 1.2511 - val_acc: 0.5152
Epoch 61/125
3948/3948 [==============================] - 1s 192us/step - loss: 0.8651 - acc: 0.6071 - val_loss: 1.3042 - val_acc: 0.4868
Epoch 62/125
3948/3948 [==============================] - 1s 192us/step - loss: 0.8435 - acc: 0.6236 - val_loss: 1.2750 - val_acc: 0.4939
Epoch 63/125
3948/3948 [==============================] - 1s 191us/step - loss: 0.8774 - acc: 0.6026 - val_loss: 1.2357 - val_acc: 0.4970
Epoch 64/125
3948/3948 [==============================] - 1s 192us/step - loss: 0.8746 - acc: 0.6016 - val_loss: 1.3751 - val_acc: 0.4676
Epoch 65/125
3948/3948 [==============================] - 1s 191us/step - loss: 0.8621 - acc: 0.6117 - val_loss: 1.3230 - val_acc: 0.5051
Epoch 66/125
3948/3948 [==============================] - 1s 190us/step - loss: 0.8587 - acc: 0.6140 - val_loss: 1.2809 - val_acc: 0.4960
Epoch 67/125
3948/3948 [=======================

In [95]:
from keras.layers import Conv2D

model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model_text.add(Convolution1D(256, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(128, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(64, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(32, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Flatten())
model_text.add(Dropout(0.2))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(125, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(512))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(256))

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(256))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech, model_mocap], mode='concat'))

model_mocap.add(Dropout(0.2))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_speech.summary()
model_text.summary()
model_mocap.summary()
model_combined.summary()

print("Model1 Built")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, padding="same", strides=(2, 2), input_shape=(200, 189,...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_32 (Flatten)         (None, 4250)              0         
_________________________________________________________________
dense_111 (Dense)            (None, 1024)              4353024   
_________________________________________________________________
activation_145 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_116 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_112 (Dense)            (None, 512)               524800    
_________________________________________________________________
activation_146 (Activation)  (None, 512)               0         
_________________________________________________________________
dropout_117 (Dropout)        (None, 512)               0         
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:57: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [96]:
hist = model_combined.fit([x_train_text,x_train_long,x_train_mocap], Y, 
                 batch_size=batch_size, nb_epoch=60, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/60
3948/3948 [==============================] - 16s 4ms/step - loss: 2.0962 - acc: 0.3361 - val_loss: 1.3285 - val_acc: 0.4231
Epoch 2/60
3948/3948 [==============================] - 13s 3ms/step - loss: 1.1580 - acc: 0.4620 - val_loss: 1.1801 - val_acc: 0.4646
Epoch 3/60
3948/3948 [==============================] - 13s 3ms/step - loss: 1.0002 - acc: 0.5646 - val_loss: 1.0507 - val_acc: 0.5719
Epoch 4/60
3948/3948 [==============================] - 13s 3ms/step - loss: 0.8115 - acc: 0.6677 - val_loss: 1.2063 - val_acc: 0.5789
Epoch 5/60
3948/3948 [==============================] - 13s 3ms/step - loss: 0.6466 - acc: 0.7419 - val_loss: 0.9470 - val_acc: 0.6731
Epoch 6/60
3948/3948 [==============================] - 13s 3ms/step - loss: 0.5015 - acc: 0.8055 - val_loss: 0.9451 - val_acc: 0.6660
Epoch 7/60
3948/3948 [==============================] - 13s 3ms/step - loss: 0.4209 - acc: 0.8419 - val_loss: 1.0361 - val_acc: 0.6721
Epoch 8/

KeyboardInterrupt: 